<a href="https://colab.research.google.com/github/CarbonicKevin/APS360-2020Summer-Project/blob/master/Model/APS360_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Model v1

##(nvm)Pull from Github

Find git token: user profile > settings > developer settings > Personal Access tokens. [Scope:](https://developer.github.com/apps/building-oauth-apps/understanding-scopes-for-oauth-apps/) repo.

In [ ]:
from google.colab import drive
from os.path import join

ROOT = '/content/drive'     # default for the drive
PROJ = 'My Drive/...'       # path to your project on Drive

GIT_USERNAME   = ""         #@param {type: "string"}
GIT_TOKEN      = ""         #@param {type: "string"}
GIT_REPOSITORY = "APS360-2020Summer-Project"


drive.mount(ROOT)           # we mount the drive at /content/drive

PROJECT_PATH = join(ROOT, PROJ)
!mkdir "{PROJECT_PATH}"I    # in case we haven't created it already   

GIT_PATH = "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git"
!mkdir ./temp
!git clone "{GIT_PATH}"
!mv ./temp/* "{PROJECT_PATH}"
!rm -rf ./temp
!rsync -aP --exclude=data/ "{PROJECT_PATH}"/*  ./

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/content/drive/My Drive/...I’: File exists
Cloning into '{GIT_REPOSITORY}'...
fatal: unable to access 'https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git/': The requested URL returned error: 400
mv: cannot stat './temp/*': No such file or directory
sending incremental file list
rsync: change_dir "/content/drive/My Drive/..." failed: No such file or directory (2)
rsync error: some files/attrs were not transferred (see previous errors) (code 23) at main.c(1196) [sender=3.1.2]


In [ ]:
# import library into google colab
import imp 
lib = imp.new_module('code/train_AI_lib.py')
exec(open("./code/train_AI_lib.py").read(), lib.__dict__)

## Import

In [ ]:
from google.colab import drive
from os.path import join

ROOT = '/content/drive'     
PROJ = 'My Drive/ENGSCI/Year 3/SUMMER 2020/APS360/Project/APS360-2020Summer-Project/'

drive.mount(ROOT)

PROJECT_PATH = join(ROOT, PROJ)

from google.colab import files

# import library

libPath = join(PROJECT_PATH,"code/")

import sys
sys.path.append(libPath)
#!cat '/content/drive/My Drive/ENGSCI/Year 3/SUMMER 2020/APS360/Project/APS360-2020Summer-Project/code/train_AI_lib.py'
from train_AI_lib import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Model

Apply openCV filters to images, store in seperate locations.

Selected OpenCV Filters:
- [Canny Edge Detection](https://docs.opencv.org/trunk/da/d22/tutorial_py_canny.html)
- [Out-of-focus Deblur Filter](https://docs.opencv.org/master/de/d3c/tutorial_out_of_focus_deblur_filter.html)
- 

[](https://colab.research.google.com/drive/1uvHuizCBqFgvbCwEhK7FvU8JW0AfxgJw#scrollTo=psH0aLrvoh78)

In [ ]:
NUM_FILTERS = 2

edgeDetect = lambda oImg: cv2.Canny(oImg, 100, 200)

func_ls = [edgeDetect #Canny edge detection
           ]

In [ ]:
imgPath = [] # image dataset
for i in range(NUM_FILTERS):
  imgPath += PROJECT_PATH + '/Data/working-wheat-data/' + str(func_ls[i])
  os.mkdir(imgPath[i])

sPath = PROJECT_PATH + "/saved/splitData/" 
os.mkdir(sPath)

iPath  = PROJECT_PATH + "/data/working-wheat-data/train"
koPath = PROJECT_PATH + "/data/working-wheat-data/train.csv"

In [ ]:
for i in range(NUM_FILTERS):
  ret = openCVImgConvert(func=func[i], oPath=imgPath[i], iPath=iPath)

Train independent models using each of the image datasets generated above.

Neural Network Architecture

In [ ]:
trainLoader, valLoader, testLoader = [], [], []

for i in range(NUM_FILTERS):
  # split data into training, validation, testing
  ret = splitData(iPath=imgPath[i], oPath=sPath, koPath=koPath)
  if (ret!=1):
    print("ERROR: splitData failed.")
    break
  
  # make dataloaders
  trainLoader_, valLoader_, testLoader_ = loadData(batchsize = 32, 
                                                   dictPath  = sPath,
                                                   inPath    = imgPath[i])
  trainLoader += [trainLoader_]
  valLoader   += [valLoader_]
  testLoader  += [testLoader_]

In [ ]:
class CNN(nn.Module):
    def __init__(self, name):
        super(CNN, self).__init__()
        self.name = name

        self.conv1 = nn.Conv2d(3,   15,  6, stride=2)  # n = 1024 -> 510
        self.conv2 = nn.Conv2d(15,  30,  6, stride=2)  # n = 510  -> 255
        self.pool1 = nn.MaxPool2d(3, 2)                # n = 255  -> 127
        self.conv3 = nn.Conv2d(30,  60,  6, stride=2)  # n = 127  -> 62
        self.pool2 = nn.MaxPool2d(4, 2)                # n = 62   -> 30

        self.fc1   = nn.Linear(29*29*60, 20)
        self.fc2   = nn.Linear(20, 1)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = self.pool2(F.relu(self.conv3(x)))
        x = x.view(-1, 29*29*60)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return(x)

In [ ]:
for i in range(NUM_FILTERS):
  # train 
  trainNet(net = CNN(),
           data = [trainLoader[i], valLoader[i]], 
           batchsize = 32, 
           epochNo = 20, 
           lr = 0.001, 
           oPath = sPath[i], 
           trainType='RegAdam', evaluate=evalRegress, isCuda=1, draw=1)